In [6]:
from pdfminer.high_level import extract_text

import pandas as pd
from copy import copy

In [7]:
import doc2pdf

doc2pdf.convert("../data/files/РыбаковСергейВячеславович_Supl.doc", '../data/files/converted_from_doc.pdf')

In [8]:
def rus(text, alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    return not alphabet.isdisjoint(text.lower())

In [9]:
def eng(text, alphabet=set('abcdefghijklmnopqrstuvwxyz')):
    return not alphabet.isdisjoint(text.lower())

In [10]:
def find_extract(text: str, text_to_find: str, bias_=0, without_new_str=False, until=None):

    idx = text.find(text_to_find) + len(text_to_find) + bias_

    list_ = []

    if until is None:
        while not(eng(text[idx])):
            if text[idx] != '\n':
                list_.append(text[idx])
            else:
                if without_new_str:
                    break
                else:
                    list_.append(' ')
            idx += 1
    else:
        while not(text[idx:].find(until) == 0):
            list_.append(text[idx])
            idx += 1
    

    return(''.join(list_).strip(' '))

In [11]:
def find_mark(text, return_index=False):

    m3 = text.find('удовлетворительно')
    m4 = text.find('хорошо')
    m5 = text.find('отлично')
    m1 = text.find('зачтено')

    min_ = min(m3, m4, m5, m1)
    if min_ == m3:
        result = 'удовлетворительно'
        if return_index:
            result = [result] + [m3]
    elif min_ == m4:
        result = 'хорошо'
        if return_index:
            result = [result] + [m4]
    elif min_ == m5:
        result = 'отлично'
        if return_index:
            result = [result] + [m5]
    else:
        result = 'зачтено'
        if return_index:
            result = [result] + [m1]

    return result

In [12]:
def parse_diploma_supplement(pdf_path):

    pdf_path = '../data/files/converted_from_doc.pdf'
    pdf_file = open(pdf_path, 'rb')
    text = extract_text(pdf_file, password='pass', page_numbers=None, maxpages=0, caching=True, codec='utf-8', laparams=None)

    general_info = pd.DataFrame([], columns=['family_name', 'date_of_birth', 'access_requirements', 'admitted_to', 'graduated_from', 'level_of_education', 'name_of_qualification', 'main_field_of_study', 'academic_programme_title', 'professional_trajectory',
                                'official_length_of_programme', 'mode_of_study', 'languages_of_instruction_examinations', 'graduation_project_theme', 'scientific_supervisor', 'graduation_project_mark', 'all_marks'])   

    subjects_list = []
    subjects_marks_begin_idx = text[text.find('Конец перечня / End of the list'):].find('\x0c') + text.find('Конец перечня / End of the list') + 2
    subjects_begin_idx = subjects_marks_begin_idx

    for i in range(3):
        marks_begin_idx = min(text[subjects_begin_idx:].find('удовлетворительно'), text[subjects_begin_idx:].find('хорошо'), text[subjects_begin_idx:].find('отлично'), text[subjects_begin_idx:].find('зачтено')) + subjects_begin_idx
        idx = marks_begin_idx
        count_n = 0
        while True:
            if text[idx] == '\n':
                count_n += 1
            idx -= 1
            if count_n >= 8:
                break
        subjects_end_idx = idx
        text1 = text[subjects_begin_idx: subjects_end_idx]
        subjects_end_idx = text1.find('\n\n') + subjects_begin_idx
        text1 = text[subjects_begin_idx: subjects_end_idx]


        subj_flag = True
        for s in text1.split('\n'):
            if subj_flag:
                if not(eng(s, alphabet=set('abcdefghjklmnopqrstuwyz'))):
                    subjects_list.append(s)
                    subj_flag = False
            else:
                if not(rus(s)):
                    subj_flag = True

        subjects_begin_idx = text[subjects_begin_idx:].find('\x0c') + subjects_begin_idx + 1

    marks_end_idx = text[subjects_marks_begin_idx:].find('Конец перечня') + subjects_marks_begin_idx
    marks_list = []

    subjects_marks_begin_idx1 = subjects_marks_begin_idx
    for i in range(len(subjects_list)):
        result = (find_mark(text[subjects_marks_begin_idx1:marks_end_idx], return_index=True))
        marks_list.append(result[0])
        subjects_marks_begin_idx1 = subjects_marks_begin_idx1 + result[1] + 1

    marks_dict = dict()
    for i, key in enumerate(subjects_list):
        marks_dict[key] = marks_list[i]

    general_info.loc[len(general_info)] = [
        find_extract(text, 'Family name(s), given name(s)'),
        text[:text.find('\n\nДата рождения (день/месяц/год)')][-10:],
        find_extract(text, 'Access requirements'),
        find_extract(text, 'Admitted to'),
        find_extract(text, 'Graduated from'),
        find_extract(text, 'Level of education'),
        find_extract(text, 'Name of qualification'),
        find_extract(text, 'Main field of study'),
        find_extract(text, 'Academic programme title'),
        find_extract(text, 'Профессиональная траектория (профессиональные траектории): '),
        find_extract(text, 'Нормативный срок освоения ', without_new_str=True),
        find_extract(text, 'Форма обучения ', without_new_str=True),
        find_extract(text, 'Language(s) of instruction /\nexaminations\n\n', without_new_str=True, until='/')[:-1],
        find_extract(text, 'Completion and defense\nof graduation project\n', until='\n\nединицы'),
        find_extract(text, 'Научный руководитель:'),
        find_mark(text[text.find('ECTS\ngrade\n\nGrade'):]),
        marks_dict
    ]

    return general_info

In [13]:
pdf_path = '../data/files/converted_from_doc.pdf'

parse_diploma_supplement(pdf_path)

,family_name,date_of_birth,access_requirements,admitted_to,graduated_from,level_of_education,name_of_qualification,main_field_of_study,academic_programme_title,professional_trajectory,official_length_of_programme,mode_of_study,languages_of_instruction_examinations,graduation_project_theme,scientific_supervisor,graduation_project_mark,all_marks
0,Рыбаков Сергей Вячеславович,05.07.2000,"Аттестат о среднем общем образовании, выданный...",Федеральное государственное бюджетное образова...,Федеральном государственном бюджетном образова...,бакалавриат,Бакалавр,01.03.02 Прикладная математика и информатика,"Прикладная математика, фундаментальная информа...",Математическое и программное обеспечение вычис...,4 года,очная,"English, Russia",Разработка масштабируемой архитектуры игрового...,кандидат технических наук И.С.Блеканов,отлично,"{'1С: Предприятие': 'отлично', 'Алгебра': 'удо..."
